In [1]:
import numpy as np
import pandas
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.regularizers import l2


In [2]:
train_data_dir = 'D:/My Documents/GIT Projects/IEEE-Hack-the-meta/ASL/asl_alphabet_train'
test_data_dir = 'D:/My Documents/GIT Projects/IEEE-Hack-the-meta/ASL/asl_alphabet_test'

In [3]:
def add_background_noise(img):
    noise_factor = 0.5
    noise = np.random.randn(*img.shape) * noise_factor
    img_noisy = img + noise
    img_noisy = np.clip(img_noisy, 0., 1.)
    return img_noisy

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    preprocessing_function=add_background_noise
)
test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=add_background_noise
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=64, 
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=64,
    class_mode='categorical'
)

Found 82650 images belonging to 29 classes.
Found 4411 images belonging to 29 classes.


In [22]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(200, 200, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(29))
model.add(Activation('softmax'))

In [23]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

In [24]:
steps_per_epoch = len(train_generator)
validation_steps = len(test_generator)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=test_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping],
)

Epoch 1/20
1156/1156 [==============================] - 1774s 2s/step - loss: 3.3676 - accuracy: 0.0331 - val_loss: 3.3673 - val_accuracy: 0.0345
Epoch 2/20
  35/1156 [..............................] - ETA: 13:51 - loss: 3.3674 - accuracy: 0.0299

KeyboardInterrupt: 

In [20]:
score = model.evaluate(test_generator, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.27153244614601135
Test accuracy: 0.9152119755744934


In [21]:
model.save('asl_model.h5')